In [2]:
from vkapi.methods import *

# Additional libs to demostrate the results
import matplotlib.pyplot as plt
%matplotlib inline

## Public methods
These methods do not require access token. Therefore they could be used without authentication